# Deep Dive: Context Engineering Dashboard

A comprehensive walkthrough of every feature in the dashboard.

**What you will learn:**

| Section | Feature |
|---|---|
| 1 | ContextResource basics |
| 2 | Chroma integration with `ContextResource.from_chroma()` |
| 3 | Resource types: RAG, EXAMPLE, CHAT_HISTORY, TOOL, SCRATCHPAD |
| 4 | Available pool visualization with `resources` parameter |
| 5 | Gesture-based interactions (hover, click, drag) |
| 6 | Live OpenAI tracing with `trace_openai()` |
| 7 | Sankey diff for prompt optimization |
| 8 | Context budget analysis |
| 9 | Serialization & reproducibility |

In [1]:
# Setup: works with pip install OR local development
import sys
from pathlib import Path

# Try importing the package - if it fails, add parent directory to path
try:
    import context_engineering_dashboard
    print(f"Using installed package: {context_engineering_dashboard.__file__}")
except ImportError:
    # Running from local clone - add parent directory to path
    repo_root = Path().resolve().parent
    if repo_root not in sys.path:
        sys.path.insert(0, str(repo_root))
    print(f"Using local development: {repo_root}")

# Installation options (run one if package not found):
# Option 1: Install from GitHub release
# !pip install git+https://github.com/cp71-dlai/context-engineering-dashboard.git@v0.1.0

# Option 2: Install locally for development (from repo root)
# !pip install -e ".[dev,all]"

Using local development: /workspace


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # reads OPENAI_API_KEY from .env

True

---
## 1 | ContextResource Basics

`ContextResource` represents a pool of items that can be selected for inclusion
in the context window. Each resource has:

- **name** - Display name for the resource pool
- **resource_type** - One of: RAG, EXAMPLE, CHAT_HISTORY, TOOL, SCRATCHPAD
- **items** - List of `ResourceItem` objects
- **selected_ids** - Set of IDs for items included in the context

In [3]:
from context_engineering_dashboard import (
    ContextResource,
    ResourceType,
    ResourceItem,
    ContextBuilder,
)

# Create a resource from a list of items
docs = ContextResource.from_items(
    items=[
        {"id": "doc_1", "content": "ChromaDB stores embeddings for semantic search.", "score": 0.95},
        {"id": "doc_2", "content": "Collections group related documents together.", "score": 0.88},
        {"id": "doc_3", "content": "Query with collection.query() to find similar docs.", "score": 0.82},
        {"id": "doc_4", "content": "Use metadata filtering with where={...} parameter.", "score": 0.75},
    ],
    resource_type=ResourceType.RAG,
    name="Documentation",
)

print(f"Resource: {docs.name}")
print(f"Type: {docs.resource_type.value}")
print(f"Items: {len(docs.items)}")
print(f"Selected: {len(docs.selected_ids)}")

Resource: Documentation
Type: rag
Items: 4
Selected: 0


In [4]:
# Selection operations
docs.select(["doc_1", "doc_2"])  # Select specific items
print(f"After select: {docs.selected_ids}")

docs.deselect(["doc_2"])  # Remove from selection
print(f"After deselect: {docs.selected_ids}")

docs.select_all()  # Select everything
print(f"After select_all: {docs.selected_ids}")

docs.clear_selection()  # Clear all
print(f"After clear: {docs.selected_ids}")

After select: {'doc_2', 'doc_1'}
After deselect: {'doc_1'}
After select_all: {'doc_3', 'doc_2', 'doc_1', 'doc_4'}
After clear: set()


In [5]:
# Convert selected items to ContextComponents
docs.select(["doc_1", "doc_3"])

components = docs.to_components()
print(f"Generated {len(components)} components:")
for c in components:
    print(f"  {c.id}: {c.type.value}, {c.token_count} tokens")

Generated 2 components:
  doc_1: rag, 9 tokens
  doc_3: rag, 10 tokens


---
## 2 | Chroma Integration

`ContextResource.from_chroma()` wraps a Chroma collection so you can query it
and manage document selection for your context window.

In [6]:
import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection(
    name="context_eng_docs",
    metadata={"description": "Context engineering reference docs"},
)

# Populate with realistic documentation chunks
doc_data = [
    {
        "id": "ce_overview",
        "text": (
            "Context engineering is the discipline of designing and optimizing the information "
            "provided to a large language model within its context window. Unlike prompt engineering, "
            "which focuses on instruction phrasing, context engineering considers the entire input: "
            "system prompts, retrieved documents, chat history, tool outputs, and user messages. "
            "The goal is to maximize the signal-to-noise ratio so the model produces accurate, "
            "grounded responses."
        ),
        "meta": {"section": "overview", "page": 1},
    },
    {
        "id": "ce_components",
        "text": (
            "A typical LLM context window contains several component types: (1) System Prompt -- "
            "defines the assistant persona and rules. (2) RAG Documents -- retrieved passages providing "
            "factual grounding. (3) Chat History -- prior turns for conversational continuity. "
            "(4) Tool Outputs -- results from function calls. (5) User Message -- the current query. "
            "Each component competes for limited token budget."
        ),
        "meta": {"section": "components", "page": 3},
    },
    {
        "id": "ce_rag_best",
        "text": (
            "RAG best practices: (1) Retrieve more than you need, then re-rank and prune. "
            "(2) Prefer smaller, focused chunks (200-400 tokens) over large passages. "
            "(3) Include metadata (source, date, score) so the model can weigh relevance. "
            "(4) Place the highest-scoring documents closest to the user query for recency bias. "
            "(5) Monitor retrieval quality with similarity-score thresholds."
        ),
        "meta": {"section": "rag", "page": 7},
    },
    {
        "id": "ce_history",
        "text": (
            "Chat history management strategies: (1) Sliding window -- keep only the last N turns. "
            "(2) Summarization -- compress older turns into a summary paragraph. "
            "(3) Selective inclusion -- only include turns relevant to the current query. "
            "(4) Token budgeting -- allocate a fixed percentage of the context window to history. "
            "Excessive history leads to distraction and increased latency."
        ),
        "meta": {"section": "history", "page": 12},
    },
    {
        "id": "ce_tools",
        "text": (
            "Tool integration patterns: Function calling lets the model invoke external APIs. "
            "Each tool definition consumes tokens from the context window. Best practices: "
            "(1) Only include tools relevant to the current task. (2) Keep descriptions concise. "
            "(3) Provide few-shot examples for complex tools. "
            "(4) Monitor tool-output token counts to avoid budget overruns."
        ),
        "meta": {"section": "tools", "page": 18},
    },
    {
        "id": "ce_eval",
        "text": (
            "Evaluating context quality: (1) Measure answer accuracy with and without specific "
            "components. (2) Track token utilization (used vs. available). (3) Compare retrieval "
            "strategies using before/after diffs. (4) Log context traces for every production call "
            "to identify patterns (e.g., which docs are never selected, which prompts are too long)."
        ),
        "meta": {"section": "evaluation", "page": 22},
    },
]

collection.add(
    ids=[d["id"] for d in doc_data],
    documents=[d["text"] for d in doc_data],
    metadatas=[d["meta"] for d in doc_data],
)

print(f"Collection '{collection.name}' has {collection.count()} documents")

onnxruntime cpuid_info warning: Unknown CPU vendor. cpuinfo_vendor value: 0


Collection 'context_eng_docs' has 6 documents


In [7]:
# Create a ContextResource from a Chroma collection
rag_resource = ContextResource.from_chroma(
    collection=collection,
    resource_type=ResourceType.RAG,
    name="Documentation",
)

# Query the resource (queries the underlying Chroma collection)
user_question = "What are the best practices for RAG in context engineering?"

rag_resource.query(
    query_texts=[user_question],
    n_results=5,
)

print(f"Query: '{user_question}'\n")
print(f"Retrieved {len(rag_resource.items)} documents:")
for item in rag_resource.items:
    print(f"  [{item.id}] score={item.score:.3f}, {item.token_count} tokens")

Query: 'What are the best practices for RAG in context engineering?'

Retrieved 5 documents:
  [ce_eval] score=0.491, 71 tokens
  [ce_components] score=0.460, 80 tokens
  [ce_overview] score=0.451, 76 tokens
  [ce_tools] score=0.426, 70 tokens
  [ce_rag_best] score=0.399, 88 tokens


In [8]:
# Select top 3 documents
top_3_ids = [item.id for item in rag_resource.items[:3]]
rag_resource.select(top_3_ids)

print(f"Selected: {list(rag_resource.selected_ids)}")
print(f"Selected tokens: {rag_resource.total_selected_tokens}")

Selected: ['ce_components', 'ce_overview', 'ce_eval']
Selected tokens: 227


---
## 3 | Resource Types

Each `ResourceType` maps to a `ComponentType` for visualization:

| ResourceType | ComponentType | Use Case |
|---|---|---|
| RAG | rag | Retrieved documents, vector search results |
| EXAMPLE | example | Few-shot Q&A pairs, demonstrations |
| CHAT_HISTORY | chat_history | Multi-turn conversation context |
| TOOL | tool | Tool/function definitions and signatures |
| SCRATCHPAD | scratchpad | User preferences, cached state, flat files |

In [9]:
# Create different resource types
examples = ContextResource.from_items(
    items=[
        {"id": "ex1", "content": "Q: What is RAG?\nA: Retrieval-Augmented Generation combines retrieval with LLMs."},
        {"id": "ex2", "content": "Q: What is embedding?\nA: A dense vector representation of text."},
    ],
    resource_type=ResourceType.EXAMPLE,
    name="Few-Shot Examples",
)
examples.select_all()

tools = ContextResource.from_items(
    items=[
        {"id": "search", "content": "search(query: str) -> List[str]: Search the knowledge base."},
        {"id": "calculate", "content": "calculate(expr: str) -> float: Evaluate a math expression."},
    ],
    resource_type=ResourceType.TOOL,
    name="Available Tools",
)
tools.select(["search"])  # Only include the search tool

print(f"Examples: {examples.name} ({len(examples.selected_ids)} selected)")
print(f"Tools: {tools.name} ({len(tools.selected_ids)} selected)")

Examples: Few-Shot Examples (2 selected)
Tools: Available Tools (1 selected)


In [10]:
# Check the component types
print(f"RAG -> {ResourceType.RAG.to_component_type().value}")
print(f"EXAMPLE -> {ResourceType.EXAMPLE.to_component_type().value}")
print(f"TOOL -> {ResourceType.TOOL.to_component_type().value}")
print(f"CHAT_HISTORY -> {ResourceType.CHAT_HISTORY.to_component_type().value}")
print(f"SCRATCHPAD -> {ResourceType.SCRATCHPAD.to_component_type().value}")

RAG -> rag
EXAMPLE -> example
TOOL -> tool
CHAT_HISTORY -> chat_history
SCRATCHPAD -> scratchpad


---
## 4 | Available Pool Visualization

When you pass `resources` to `ContextBuilder`, it shows a two-panel view:
- **Left panel**: All available items from each resource
- **Right panel**: Selected items included in the context window

In [11]:
from context_engineering_dashboard import ContextTrace, ContextComponent, ComponentType

# Build a trace with system prompt and user message
base_components = [
    ContextComponent(
        "sys",
        ComponentType.SYSTEM_PROMPT,
        "You are an expert on context engineering. Answer using only the provided documents.",
        token_count=20,
    ),
    ContextComponent(
        "user",
        ComponentType.USER_MESSAGE,
        user_question,
        token_count=15,
    ),
]

# Add selected RAG components
all_components = base_components + rag_resource.to_components()

trace = ContextTrace(
    context_limit=128_000,
    components=all_components,
    total_tokens=sum(c.token_count for c in all_components),
)

print(f"Trace has {len(trace.components)} components, {trace.total_tokens} tokens")

Trace has 5 components, 262 tokens


In [ ]:
# Visualize with resources panel
# Left: Available items (all retrieved docs)
# Right: Context window (system + user + selected docs)
temp = ContextBuilder(trace=trace, resources=[])
temp.

In [15]:
# Combine multiple resources
ContextBuilder(trace=trace, resources=[rag_resource, examples, tools])

---
## 5 | Gesture-based Interactions

The dashboard uses intuitive mouse gestures:

| Gesture | Action |
|---------|--------|
| **Hover** | Tooltip showing component type and token count |
| **Click** | Opens a modal with full content and metadata |
| **Click text in modal** | Switch to edit mode (Save button appears) |
| **Drag** | Move items between Available and Context panels |

In [14]:
# Try the gestures on this visualization:
# - Hover over any block to see the tooltip
# - Click a component to see its full content and metadata
# - Click on the text content to edit (Save button appears)
# - Drag items between panels to change selection
ContextBuilder(trace=trace, resources=[rag_resource])

---
## 6 | Live OpenAI Tracing

`trace_openai()` monkey-patches the OpenAI client to capture every
`chat.completions.create` call inside the `with` block. Token counts
come from the API's own usage data.

In [17]:
from openai import OpenAI
from context_engineering_dashboard import trace_openai

client = OpenAI()

system_prompt = (
    "You are an expert on context engineering for large language models. "
    "When answering, structure your response with clear headings and examples."
)

with trace_openai() as tracer:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": (
                "Compare three chat-history management strategies for LLM context windows: "
                "sliding window, summarization, and selective inclusion. "
                "Include trade-offs for each."
            )},
        ],
        temperature=0.7,
    )

print(response.choices[0].message.content)

When managing chat history for large language models (LLMs) with limited context windows, it's crucial to employ effective strategies to maximize the relevance and utility of the given context. Here, I compare three strategies: sliding window, summarization, and selective inclusion, highlighting their trade-offs.

### 1. Sliding Window Strategy

**Description:**
The sliding window strategy involves maintaining a fixed-size window that moves through the conversation as it progresses. Only the most recent exchanges are kept within the context window, and older messages are discarded as new ones come in.

**Trade-offs:**
- **Pros:**
  - **Simplicity:** This method is straightforward to implement, requiring minimal computational overhead.
  - **Recency:** Always focuses on the most recent interactions, which are often the most relevant.

- **Cons:**
  - **Loss of Context:** Older, potentially important messages are lost, which can result in a loss of critical context for understanding ongo

In [18]:
openai_trace = tracer.result

print(f"Model:       {openai_trace.trace.model}")
print(f"Provider:    {openai_trace.trace.provider}")
print(f"Prompt:      {openai_trace.trace.usage.get('prompt_tokens', '?')} tokens")
print(f"Completion:  {openai_trace.trace.usage.get('completion_tokens', '?')} tokens")
print(f"Latency:     {openai_trace.trace.latency_ms:.0f} ms")
print(f"Context:     {openai_trace.context_limit:,} tokens")

# Click any component to see details
ContextBuilder(trace=openai_trace)

Model:       gpt-4o
Provider:    openai
Prompt:      63 tokens
Completion:  730 tokens
Latency:     15831 ms
Context:     128,000 tokens


---
## 7 | Sankey Diff for Prompt Optimization

`ContextDiff` renders an SVG Sankey diagram showing how token allocation
changed between two versions of a prompt.

In [19]:
from context_engineering_dashboard import ContextDiff

# Scenario: optimize a production RAG prompt
before = ContextTrace(
    context_limit=128_000,
    components=[
        ContextComponent("sys",    ComponentType.SYSTEM_PROMPT, "...", token_count=5000),
        ContextComponent("rag_1",  ComponentType.RAG,  "...", token_count=8000),
        ContextComponent("rag_2",  ComponentType.RAG,  "...", token_count=6000),
        ContextComponent("rag_3",  ComponentType.RAG,  "...", token_count=4000),
        ContextComponent("hist",   ComponentType.CHAT_HISTORY,  "...", token_count=22000),
        ContextComponent("tools",  ComponentType.TOOL,          "...", token_count=3000),
        ContextComponent("user",   ComponentType.USER_MESSAGE,  "...", token_count=500),
    ],
    total_tokens=48500,
)

# After optimization:
# - Summarized chat history (22K -> 5K)
# - Dropped lowest-scoring RAG doc
# - Trimmed system prompt
after = ContextTrace(
    context_limit=128_000,
    components=[
        ContextComponent("sys",    ComponentType.SYSTEM_PROMPT, "...", token_count=2500),
        ContextComponent("rag_1",  ComponentType.RAG,  "...", token_count=8000),
        ContextComponent("rag_2",  ComponentType.RAG,  "...", token_count=6000),
        ContextComponent("hist",   ComponentType.CHAT_HISTORY,  "...", token_count=5000),
        ContextComponent("tools",  ComponentType.TOOL,          "...", token_count=3000),
        ContextComponent("user",   ComponentType.USER_MESSAGE,  "...", token_count=500),
    ],
    total_tokens=25000,
)

diff = ContextDiff(
    before=before,
    after=after,
    before_label="v1 (Verbose)",
    after_label="v2 (Optimized)",
)

diff.sankey()

In [20]:
diff.summary()

Component                Before      After     Change
----------------------------------------------------
History                  22,000      5,000       -77%
RAG                      18,000     14,000       -22%
System                    5,000      2,500       -50%
Tool                      3,000      3,000        +0%
User                        500        500        +0%
----------------------------------------------------
Total                    48,500     25,000    -23,500

Saved 23,500 tokens (48.5% reduction)


In [21]:
# Side by side: before and after
from IPython.display import display, HTML

display(HTML("<h3>BEFORE</h3>"))
display(ContextBuilder(trace=before))
display(HTML("<h3>AFTER</h3>"))
display(ContextBuilder(trace=after))

---
## 8 | Context Budget Analysis

Programmatically inspect where your tokens go.

In [22]:
# Analyze the "before" trace from Section 7
analysis_trace = before

print("CONTEXT BUDGET ANALYSIS")
print("=" * 50)
print(f"Context limit:  {analysis_trace.context_limit:>10,} tokens")
print(f"Tokens used:    {analysis_trace.total_tokens:>10,} tokens")
print(f"Tokens free:    {analysis_trace.unused_tokens:>10,} tokens")
print(f"Utilization:    {analysis_trace.utilization:>9.1f}%")
print()
print(f"{'Component Type':<22} {'Tokens':>8} {'% of Used':>10} {'Count':>6}")
print("-" * 50)

for comp_type in ComponentType:
    comps = analysis_trace.get_components_by_type(comp_type)
    if comps:
        total = sum(c.token_count for c in comps)
        pct = (total / analysis_trace.total_tokens) * 100 if analysis_trace.total_tokens else 0
        print(f"  {comp_type.value:<20} {total:>8,} {pct:>9.1f}% {len(comps):>5}")

CONTEXT BUDGET ANALYSIS
Context limit:     128,000 tokens
Tokens used:        48,500 tokens
Tokens free:        79,500 tokens
Utilization:         37.9%

Component Type           Tokens  % of Used  Count
--------------------------------------------------
  system_prompt           5,000      10.3%     1
  user_message              500       1.0%     1
  chat_history           22,000      45.4%     1
  rag                    18,000      37.1%     3
  tool                    3,000       6.2%     1


In [23]:
# Individual component breakdown
print(f"\n{'ID':<20} {'Type':<18} {'Tokens':>7}")
print("-" * 50)
for c in analysis_trace.components:
    print(f"{c.id:<20} {c.type.value:<18} {c.token_count:>7,}")


ID                   Type                Tokens
--------------------------------------------------
sys                  system_prompt        5,000
rag_1                rag                  8,000
rag_2                rag                  6,000
rag_3                rag                  4,000
hist                 chat_history        22,000
tools                tool                 3,000
user                 user_message           500


---
## 9 | Serialization & Reproducibility

Traces serialize to JSON so you can save snapshots, share with teammates,
and compare across time.

In [24]:
import json

# Save to JSON
trace.to_json("deep_dive_trace.json")
print("Saved trace to deep_dive_trace.json")

# Peek at the JSON structure
with open("deep_dive_trace.json") as f:
    data = json.load(f)

print(f"\nJSON keys: {list(data.keys())}")
print(f"Components: {len(data['components'])}")
print(f"Total tokens: {data['total_tokens']:,}")

Saved trace to deep_dive_trace.json

JSON keys: ['context_limit', 'total_tokens', 'components', 'trace', 'embedding_traces', 'timestamp', 'session_id', 'tags']
Components: 5
Total tokens: 262


In [25]:
# Reload and visualize
reloaded = ContextTrace.from_json("deep_dive_trace.json")

print(f"Reloaded: {len(reloaded.components)} components, {reloaded.total_tokens:,} tokens")

ContextBuilder(trace=reloaded)

Reloaded: 5 components, 262 tokens


In [26]:
# Convert to dict for programmatic access
trace_dict = trace.to_dict()

# Example: extract all RAG scores
for comp in trace_dict["components"]:
    if comp["type"] == "rag" and comp.get("metadata"):
        score = comp["metadata"].get("score", "N/A")
        print(f"  {comp['id']}: score={score}")

  ce_eval: score=0.4909
  ce_components: score=0.4597
  ce_overview: score=0.4508


In [27]:
# Clean up
os.remove("deep_dive_trace.json")

---

## Summary

| Feature | How to use it |
|---|---|
| ContextResource | `ContextResource.from_items(items, resource_type, name)` |
| Chroma integration | `ContextResource.from_chroma(collection, resource_type, name)` then `.query()` |
| Selection | `resource.select([ids])`, `resource.deselect([ids])`, `resource.select_all()` |
| Convert to components | `resource.to_components()` |
| OpenAI tracing | `with trace_openai() as t:` wraps any `client.chat.completions.create` call |
| Visualization | `ContextBuilder(trace=trace)` |
| Available pool | `ContextBuilder(trace=trace, resources=[resource1, resource2])` |
| Interactions | Hover (tooltip), Click (modal), Click text (edit), Drag (move items) |
| Diff view | `ContextDiff(before, after).sankey()` |
| Save/load | `trace.to_json('file.json')` / `ContextTrace.from_json('file.json')` |
| Budget analysis | `trace.utilization`, `trace.unused_tokens`, `trace.get_components_by_type()` |